In [313]:
from dataclasses import dataclass
from typing import Union
from datetime import datetime
from usosapi import USOSAPIConnection
from webbrowser import open
from functools import reduce

# Logging in
Firstly, we are creating new connection using consumer key and secret. Then, user need to perform authorization via browser and insert given pin number.

In [3]:
connection = USOSAPIConnection(api_base_address='https://apps.usos.pwr.edu.pl/',
                               consumer_key='ssijmi',
                               consumer_secret='chuja')
connection_url = connection.get_authorization_url()
open(connection_url)
pin = input()

KeyError: 'Decoder failed to handle oauth_token with data as returned by provider. A different decoder may be needed. Provider returned: b\'{"message": "Invalid consumer."}\''

USOSapi creates connection and test it.

In [3]:
connection.authorize_with_pin(pin)
connection.is_authorized()

True

Now we are able to download logged-in user's registrations.

<div class="alert alert-block alert-warning">
<b>Warning:</b> We are not sure how will list behave when new registrations will appear. We may introduce choose of right registrations.
</div>

Downloading and picking courses' ids.

# EXAMPLE
Picking one course. I've chose Database design with Nguyen because the lecture is presented once every two weeks.

In [458]:
from planner.models.groups import WeekType


@dataclass
class Course_type:
    id: int
    name: str
    

@dataclass
class Lecturer:
    id: int
    first_name: str
    last_name: str

@dataclass
class Course_group:
    """
    usos: COURSE EDITION
    enrollment assistant: GRUPA KURSÓW
    """
    name: str
    code: str
    
    
@dataclass
class Course:
    """
    usos: COURSE UNIT
    enrollment assistant: KURS
    """
    type: Course_type
    course_group: Course_group

@dataclass
class Group:
    """
    usos: GROUP / CLASS GROUP
    enrollment assistant: GRUPA ZAJĘCIOWA
    """
    lecturers: list[Lecturer]
    course: Course
    weekday: int
    week_type: WeekType
    start_time: datetime
    end_time: datetime

In [459]:
def download_course_types() -> dict:
    return connection.get('services/courses/classtypes_index')

def get_course_types() -> list[Course_type]:
    types: list[Course_type] = []
    for course_type in download_course_types().values():
        new_type = Course_type(course_type['id'], course_type['name']['pl'])
        types.append(new_type)
    return types

def get_course_groups(registration_id: str) -> list[Course_group]:
    syllabus_courses = get_syllabus_courses(registration_id)
    return [create_course_group(course_dict) for course_dict in syllabus_courses]
    
def get_syllabus_courses(registration_id: str):
    syllabus_courses_ids = download_syllabus_courses_ids(registration_id)
    query_from_syllabus = reduce(lambda x, y: x + '|' + y, syllabus_courses_ids)
    syllabus_courses = download_syllabus_courses(query_from_syllabus)
    return list(syllabus_courses.values())

def download_syllabus_courses_ids(registration_id: str) -> list[dict]:
    return [record['course_id'] for record in connection.get('services/registrations/registration', id=registration_id, fields='related_courses')['related_courses']]

def download_syllabus_courses(ids_query_str: str) -> list[dict]:
    return connection.get('services/courses/courses', fields='id|name', course_ids=ids_query_str)

def create_course_group(course_dict) -> Course_group:
    return Course_group(code=course_dict['id'], name=course_dict['name']['pl'])

def get_group_dates(unit_id, group_number):
    return convert_to_dates(get_group_activities(unit_id,group_number))

def get_group_activities(unit_id, group_number):
    return connection.get('services/tt/classgroup_dates', unit_id=unit_id, group_number=group_number)

def convert_to_dates(group_activities):
    return [date.fromisoformat(day) for day in group_activities]

def get_weekday(group_dates):
    return Counter([DayOfWeek(date.weekday()+1) for date in group_dates]).most_common()[0][0]

def get_week_type(group_dates):
    if len(dates) > 10:
        return WeekType.EVERY_WEEK
    else:
        counter = Counter([date.isocalendar()[1] % 2 for date in group_dates]).most_common()
        if len(counter) == 1:
            return WeekType(counter[0] + 2)
        elif counter[0][1] - counter[1][1] > 4:
            return WeekType(counter[0] + 2)
        else:
            return WeekType.OTHER

In [460]:
connection.get('services/courses/course_edition2', course_id=course_id, term_id=term, fields='course_units[id|class_groups[lecturers]]')

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
get_syllabus_courses(ist_summer_registration_id)

# MAIN FLOW

In [461]:
ist_summer_registration_id = 'W04-IST-SI-6-23/24L'
term='2023/24-L'
types = get_course_types()
course_groups = get_course_groups(ist_summer_registration_id)
for crs_group in course_groups:
    print(crs_group)
    units = connection.get('services/courses/course_edition2', course_id=crs_group.code, term_id=term, fields='course_units[id|class_groups[number|lecturers]]')['course_units']
    lecturers = {}
    for course in units:
        new_course = None
        for group in course['class_groups']:
            print(group)
            days = get_group_activities(course['id'],group['number'])
            dates = convert_to_dates(days)
            weekday = get_weekday(dates)
            week_type = get_week_type(dates)
            group_lecturers = []
            for lecturer in group['lecturers']:
                if lecturer['id'] not in lecturers.keys():
                    lecturers[lecturer['id']] = Lecturer(lecturer['id'], lecturer['first_name'], lecturer['last_name'])
                group_lecturers.append(lecturers[lecturer['id']])
            new_group = Group(lecturers=group_lecturers, course=new_course, weekday=weekday, week_type=week_type, start_time=None, end_time=None)
            print(group)
        print()


Course_group(name='Hurtownie danych', code='W04IST-SI0012L')
{'number': 1, 'lecturers': [{'id': '74307', 'first_name': 'Bernadetta', 'last_name': 'Maleszka'}]}
{'number': 1, 'lecturers': [{'id': '74307', 'first_name': 'Bernadetta', 'last_name': 'Maleszka'}]}
{'number': 2, 'lecturers': [{'id': '187', 'first_name': 'Lech', 'last_name': 'Tuzinkiewicz'}]}
{'number': 2, 'lecturers': [{'id': '187', 'first_name': 'Lech', 'last_name': 'Tuzinkiewicz'}]}
{'number': 3, 'lecturers': [{'id': '74307', 'first_name': 'Bernadetta', 'last_name': 'Maleszka'}]}
{'number': 3, 'lecturers': [{'id': '74307', 'first_name': 'Bernadetta', 'last_name': 'Maleszka'}]}
{'number': 4, 'lecturers': [{'id': '74307', 'first_name': 'Bernadetta', 'last_name': 'Maleszka'}]}
{'number': 4, 'lecturers': [{'id': '74307', 'first_name': 'Bernadetta', 'last_name': 'Maleszka'}]}
{'number': 5, 'lecturers': [{'id': '74307', 'first_name': 'Bernadetta', 'last_name': 'Maleszka'}]}
{'number': 5, 'lecturers': [{'id': '74307', 'first_name'

TypeError: can only concatenate tuple (not "int") to tuple

In [ ]:
connection.get('services/courses/course_edition2', course_id=crs_group.code, term_id=term, fields='course_units[id|class_groups[lecturers]]')

In [ ]:
courses = [record['course_id'] for record in connection.get('services/registrations/registration', id='W04-IST-SI-6-23/24L', fields='related_courses')['related_courses']]

In [135]:
course_id = courses[9]
connection.get('services/courses/course', course_id=course_id)

{'id': 'W04IST-SI0824G',
 'name': {'pl': 'Programowanie aplikacji multimedialnych',
  'en': 'Program Multimedia Applications'}}

Getting term to use new API method, *services/courses/course'*.

In [387]:
term = connection.get('services/courses/course', course_id=course_id, fields='terms')['terms'][0]['id']

In [388]:
term

'2023/24-L'

In this term course's edition we have lecture and project classes.

We pick the first one. Fun fact: we are able to see if this chose is right in the next steps, empirically, not yet.

In [138]:
unit = units[0]['id']

In [139]:
unit

'65724'

In [363]:
connection.get('services/courses/unit',unit_id=unit, )

{'id': '65724'}

Now, look at the unit's groups.

In [141]:
class_groups = connection.get('services/courses/course_unit',course_unit_id=unit, fields='class_groups')['class_groups']

In [142]:
class_groups

[{'course_unit_id': '65724', 'number': 1},
 {'course_unit_id': '65724', 'number': 2},
 {'course_unit_id': '65724', 'number': 3},
 {'course_unit_id': '65724', 'number': 4},
 {'course_unit_id': '65724', 'number': 5},
 {'course_unit_id': '65724', 'number': 6}]

Hurrah. If it has only one group, it is the lecture - we've chosen right unit.

In [143]:
connection.get('services/groups/class_group',course_unit_id=class_groups[0]['course_unit_id'],              group_number=class_groups[0]['number'], fields='lecturers')

{'lecturers': [{'id': '344836',
   'first_name': 'Katarzyna',
   'last_name': 'Białas'}]}

Next fun fact: there is no **Than** between Ngoc and Nguyen, because programmers didn't create place for the second name. Bad business analysis?
And then we are able to look into dates of each of lectures.

In [144]:
connection.get('services/tt/classgroup_dates2',
               unit_id=class_groups[0]['course_unit_id'],
               group_number=class_groups[0]['number'], 
               fields='start_time|end_time|name|cgwm_id|classtype_id|unit_id')

[{'start_time': '2024-03-01 17:05:00',
  'end_time': '2024-03-01 18:45:00',
  'name': {'pl': 'Zajęcia laboratoryjne', 'en': 'Laboratory'},
  'cgwm_id': 47506,
  'classtype_id': 'L',
  'unit_id': 65724},
 {'start_time': '2024-03-08 17:05:00',
  'end_time': '2024-03-08 18:45:00',
  'name': {'pl': 'Zajęcia laboratoryjne', 'en': 'Laboratory'},
  'cgwm_id': 47506,
  'classtype_id': 'L',
  'unit_id': 65724},
 {'start_time': '2024-03-15 17:05:00',
  'end_time': '2024-03-15 18:45:00',
  'name': {'pl': 'Zajęcia laboratoryjne', 'en': 'Laboratory'},
  'cgwm_id': 47506,
  'classtype_id': 'L',
  'unit_id': 65724},
 {'start_time': '2024-03-22 17:05:00',
  'end_time': '2024-03-22 18:45:00',
  'name': {'pl': 'Zajęcia laboratoryjne', 'en': 'Laboratory'},
  'cgwm_id': 47506,
  'classtype_id': 'L',
  'unit_id': 65724},
 {'start_time': '2024-04-05 17:05:00',
  'end_time': '2024-04-05 18:45:00',
  'name': {'pl': 'Zajęcia laboratoryjne', 'en': 'Laboratory'},
  'cgwm_id': 47506,
  'classtype_id': 'L',
  'uni

# Question
##### What do we do now?
I did not find any simpler methods to extract data about odd/even weeks and day of the week. Therefore, if there is no such method, we must derive necessary data from lists such as this one above, with start_time and end_time and only these. It is not very hard task, nevertheless **how** to conduct such translation of information is open. Buzi.

In addition: we may use special methods to request for many entities at once.

In [145]:
courses[:3]

['W04IST-SI0012L', 'W04IST-SI0012W', 'W04IST-SI0023L']

In [353]:
course_ids = reduce(lambda x, y: x + '|' + y, courses)
connection.get('services/courses/courses', fields='id|name|ects_credits_simplified', course_ids=course_ids)

{'W04IST-SI0012L': {'id': 'W04IST-SI0012L',
  'name': {'pl': 'Hurtownie danych', 'en': 'Data Warehouses'},
  'ects_credits_simplified': 2.0},
 'W04IST-SI0012W': {'id': 'W04IST-SI0012W',
  'name': {'pl': 'Hurtownie danych', 'en': 'Data Warehouses'},
  'ects_credits_simplified': 2.0},
 'W04IST-SI0023L': {'id': 'W04IST-SI0023L',
  'name': {'pl': 'Sztuczna inteligencja i inżynieria wiedzy',
   'en': 'Artificial Intelligence and Knowledge Engineering'},
  'ects_credits_simplified': 3.0},
 'W04IST-SI0023W': {'id': 'W04IST-SI0023W',
  'name': {'pl': 'Sztuczna inteligencja i inżynieria wiedzy',
   'en': 'Artificial Intelligence and Knowledge Engineering'},
  'ects_credits_simplified': 2.0},
 'W04IST-SI0801G': {'id': 'W04IST-SI0801G',
  'name': {'pl': 'Wprowadzenie do zarządania projektami informatycznymi',
   'en': 'Introduction to IT Project Management'},
  'ects_credits_simplified': 4.0},
 'W04IST-SI0802G': {'id': 'W04IST-SI0802G',
  'name': {'pl': 'Wspomaganie zarządzania projektami informa

In [346]:
connection.get('services/courses/course_edition', course_id='W04IST-SI0023L', term_id=term, fields='course_units_ids')

{'course_units_ids': ['62146']}

In [318]:
from collections import Counter
from planner.models.groups import DayOfWeek, WeekType, Group
from datetime import date, datetime

In [305]:
def get_groups(course_unit_id):
    class_groups = connection.get('services/courses/course_unit', course_unit_id=course_unit_id, fields='class_groups')['class_groups']
    groups_ids = reduce(lambda x, y: x + '|(' + y['course_unit_id'] + ',' + str(y['number']) + ')', class_groups, '')[1:]
    groups = connection.get('services/groups/groups', group_ids=groups_ids, fields='lecturers')
    return groups

In [339]:
from typing import Dict


def get_unit_groups(course_unit_id: Union[str,int]) -> list[Dict[str, str]]:
    unit_groups = connection.get('services/courses/course_unit', course_unit_id=course_unit_id, fields='class_groups')['class_groups']
    return unit_groups    

In [340]:
unit_groups

[{'course_unit_id': '62146', 'number': 1},
 {'course_unit_id': '62146', 'number': 2},
 {'course_unit_id': '62146', 'number': 3},
 {'course_unit_id': '62146', 'number': 4},
 {'course_unit_id': '62146', 'number': 5},
 {'course_unit_id': '62146', 'number': 6},
 {'course_unit_id': '62146', 'number': 7},
 {'course_unit_id': '62146', 'number': 8},
 {'course_unit_id': '62146', 'number': 9},
 {'course_unit_id': '62146', 'number': 10},
 {'course_unit_id': '62146', 'number': 11}]

In [337]:
unit_id = 62146
unit_groups = get_unit_groups(62146)
groups = []
for group in unit_groups:
    group_dates = get_group_dates(unit_id, group['number'])
    group_weekday = get_day(group_dates)
    group_week_type = get_week_type(group_dates)
    new_group = (group_week_type,group_weekday)
    groups.append(new_group)

In [338]:
groups

[(<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Monday: 1>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Monday: 1>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Tuesday: 2>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Friday: 5>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Tuesday: 2>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Friday: 5>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Friday: 5>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Thursday: 4>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Friday: 5>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Tuesday: 2>),
 (<WeekType.EVERY_WEEK: 1>, <DayOfWeek.Friday: 5>)]

In [341]:
get_groups('62146')

{'(62146,1)': {'lecturers': [{'id': '342626',
    'first_name': 'Julita',
    'last_name': 'Bielaniewicz',
    'user_id': '342626'}]},
 '(62146,2)': {'lecturers': [{'id': '342626',
    'first_name': 'Julita',
    'last_name': 'Bielaniewicz',
    'user_id': '342626'}]},
 '(62146,3)': {'lecturers': [{'id': '346079',
    'first_name': 'Katarzyna',
    'last_name': 'Fojcik',
    'user_id': '346079'}]},
 '(62146,4)': {'lecturers': [{'id': '405702',
    'first_name': 'Stanisław',
    'last_name': 'Woźniak',
    'user_id': '405702'}]},
 '(62146,5)': {'lecturers': [{'id': '188637',
    'first_name': 'Przemysław',
    'last_name': 'Dolata',
    'user_id': '188637'}]},
 '(62146,6)': {'lecturers': [{'id': '341661',
    'first_name': 'Daria',
    'last_name': 'Dziubałtowska',
    'user_id': '341661'}]},
 '(62146,7)': {'lecturers': [{'id': '444081',
    'first_name': 'Nadezhda',
    'last_name': 'Dazhunts',
    'user_id': '444081'}]},
 '(62146,8)': {'lecturers': [{'id': '412556',
    'first_name': 

# Z TEGO WYSZŁO TAMTO:

In [ ]:
class_groups = connection.get('services/courses/course_unit', course_unit_id='62146', fields='class_groups')['class_groups']

In [227]:
connection.get('services/tt/classgroup', unit_id='62146', start='2024-03-11', group_number='6', fields='name|cgwm_id|lecturer_ids|course_name|classtype_name|frequency')

[{'name': {'pl': 'Zajęcia laboratoryjne', 'en': 'Laboratory'},
  'cgwm_id': 47441,
  'lecturer_ids': [341661],
  'course_name': {'pl': 'Sztuczna inteligencja i inżynieria wiedzy',
   'en': 'Artificial Intelligence and Knowledge Engineering'},
  'classtype_name': {'pl': 'Zajęcia laboratoryjne', 'en': 'Laboratory'}}]

In [293]:
days = get_group_activities('62146','6')
dates = convert_to_dates(days)
get_day(dates)
get_week_type(dates)

<WeekType.EVERY_WEEK: 1>

In [250]:
date.fromisoformat('2024-01-01')

datetime.date(2024, 1, 1)

In [264]:
Counter([DayOfWeek(date.weekday()+1) for date in dates]).most_common()[0][0]

<DayOfWeek.Friday: 5>

In [275]:
get_week_type(dates)

<WeekType.EVERY_WEEK: 1>

In [271]:
Counter([date.isocalendar()[1] % 2 for date in dates]).most_common()

[(1, 8), (0, 7)]

In [202]:
groups_ids

'(62146,1)|(62146,2)|(62146,3)|(62146,4)|(62146,5)|(62146,6)|(62146,7)|(62146,8)|(62146,9)|(62146,10)|(62146,11)'

In [203]:
connection.get('services/groups/groups', group_ids=groups_ids, fields='lecturers')

{'(62146,1)': {'lecturers': [{'id': '342626',
    'first_name': 'Julita',
    'last_name': 'Bielaniewicz',
    'user_id': '342626'}]},
 '(62146,2)': {'lecturers': [{'id': '342626',
    'first_name': 'Julita',
    'last_name': 'Bielaniewicz',
    'user_id': '342626'}]},
 '(62146,3)': {'lecturers': [{'id': '346079',
    'first_name': 'Katarzyna',
    'last_name': 'Fojcik',
    'user_id': '346079'}]},
 '(62146,4)': {'lecturers': [{'id': '405702',
    'first_name': 'Stanisław',
    'last_name': 'Woźniak',
    'user_id': '405702'}]},
 '(62146,5)': {'lecturers': [{'id': '188637',
    'first_name': 'Przemysław',
    'last_name': 'Dolata',
    'user_id': '188637'}]},
 '(62146,6)': {'lecturers': [{'id': '341661',
    'first_name': 'Daria',
    'last_name': 'Dziubałtowska',
    'user_id': '341661'}]},
 '(62146,7)': {'lecturers': [{'id': '444081',
    'first_name': 'Nadezhda',
    'last_name': 'Dazhunts',
    'user_id': '444081'}]},
 '(62146,8)': {'lecturers': [{'id': '412556',
    'first_name': 